An easy CNN network for digital classification.Author SUNNK.Time 2017/7/22.

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
learning_rate = 0.0001
training_iters = 2000000
batch_size = 128
display_step = 100

n_input = 784 
n_classes = 10 
dropout = 0.75  #dropout是指在深度学习网络的训练过程中，对于神经网络单元，按照一定的概率将其暂时从网络中丢弃。注意是暂时，
                            #对于随机梯度下降来说，由于是随机丢弃，故而每一个mini-batch都在训练不同的网络

x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) 

In [8]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)

    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [9]:
weights = { #关于卷积神经网络参数计算问题可以参考http://blog.csdn.net/dcxhun3/article/details/46878999
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = conv_net(x, weights, biases, keep_prob)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [11]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("迭代次数： " + str(step*batch_size) + ", 损失= " + \
                  "{:.6f}".format(loss) + ", 训练准确率= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("训练完成")

    print("测试准确率:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

迭代次数： 12800, 损失= 20697.851562, 训练准确率= 0.30469
迭代次数： 25600, 损失= 12963.962891, 训练准确率= 0.42188
迭代次数： 38400, 损失= 6155.549316, 训练准确率= 0.67969
迭代次数： 51200, 损失= 6231.414062, 训练准确率= 0.75781
迭代次数： 64000, 损失= 3723.636230, 训练准确率= 0.83594
迭代次数： 76800, 损失= 2679.205078, 训练准确率= 0.84375
迭代次数： 89600, 损失= 3563.668945, 训练准确率= 0.86719
迭代次数： 102400, 损失= 3190.264893, 训练准确率= 0.84375
迭代次数： 115200, 损失= 2269.433594, 训练准确率= 0.85938
迭代次数： 128000, 损失= 1687.265137, 训练准确率= 0.88281
迭代次数： 140800, 损失= 2120.260986, 训练准确率= 0.89062
迭代次数： 153600, 损失= 2463.311035, 训练准确率= 0.88281
迭代次数： 166400, 损失= 2155.946777, 训练准确率= 0.90625
迭代次数： 179200, 损失= 900.847290, 训练准确率= 0.90625
迭代次数： 192000, 损失= 4498.732422, 训练准确率= 0.86719
迭代次数： 204800, 损失= 957.868042, 训练准确率= 0.95312
迭代次数： 217600, 损失= 2042.963379, 训练准确率= 0.89062
迭代次数： 230400, 损失= 1005.500305, 训练准确率= 0.92969
迭代次数： 243200, 损失= 542.450439, 训练准确率= 0.95312
迭代次数： 256000, 损失= 1956.842773, 训练准确率= 0.88281
迭代次数： 268800, 损失= 1372.397217, 训练准确率= 0.93750
迭代次数： 281600, 损失= 970.156677, 训练准确率= 0.929